In [5]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Day6Spark").master("local").getOrCreate()

##### Simple way to change the GENDER column with appropriate values based on available valu in columns

that is if the column has value as "Female", the new value will be "F"
and if the column has value as "Male", the new value will be "M".
If there is No value in column we will set the value to "Not available"

For this we are using when() and otherwise functions.

PySpark when() is SQL function, in order to use this first you should import and this returns a Column type, otherwise() is a function of Column, when otherwise() not used and none of the conditions met it assigns None (Null) value. 

Usage would be like when(condition).otherwise(default).

when() function take 2 parameters, first param takes a condition and second takes a literal value or Column, if condition evaluates to true then it returns a value from second param.

In [15]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1), \
       (6,"Vishu",2,"2010","50","Male",-1),\
       (6,"Mayuri",2,"2010","50","Female",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]

empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)


root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
|6     |Vishu   |2              |2010       |50         |Male  |-1   

In [14]:
from  pyspark.sql.functions import *
empDF.withColumn("new_gender", \
                 when(col("gender")=="Male","M")\
                 .when(col("gender")== "Female","F")\
                 .when(col("gender")== "","Not Available")
                 .otherwise(empDF.gender)).show()

+------+--------+---------------+-----------+-----------+------+------+-------------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|   new_gender|
+------+--------+---------------+-----------+-----------+------+------+-------------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|            M|
|     2|    Rose|              1|       2010|         20|     M|  4000|            M|
|     3|Williams|              1|       2010|         10|     M|  1000|            M|
|     4|   Jones|              2|       2005|         10|     F|  2000|            F|
|     5|   Brown|              2|       2010|         40|      |    -1|Not Available|
|     6|   Brown|              2|       2010|         50|      |    -1|Not Available|
|     6|   Vishu|              2|       2010|         50|  Male|    -1|            M|
|     6|  Mayuri|              2|       2010|         50|Female|    -1|            F|
+------+--------+---------------+-----------+---------